In [1]:
"""
    SORT: A Simple, Online and Realtime Tracker
    Copyright (C) 2016-2020 Alex Bewley alex@bewley.ai

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""

'\n    SORT: A Simple, Online and Realtime Tracker\n    Copyright (C) 2016-2020 Alex Bewley alex@bewley.ai\n\n    This program is free software: you can redistribute it and/or modify\n    it under the terms of the GNU General Public License as published by\n    the Free Software Foundation, either version 3 of the License, or\n    (at your option) any later version.\n\n    This program is distributed in the hope that it will be useful,\n    but WITHOUT ANY WARRANTY; without even the implied warranty of\n    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the\n    GNU General Public License for more details.\n\n    You should have received a copy of the GNU General Public License\n    along with this program.  If not, see <http://www.gnu.org/licenses/>.\n'

In [2]:
# matplotlib.use('TkAgg')

In [13]:
from __future__ import print_function

import os
import numpy as np
import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

import glob
import time
import argparse
from filterpy.kalman import KalmanFilter

np.random.seed(0)

In [14]:
# input : cost_matrix, output : 매칭(할당). 원소가 [row_ind, col_ind]로 이루어진 2차원 np.array
# lapjv : 헝가리안 알고리즘보다 훨씬 빠른 Jonker-Volgenant Algorithm 사용
# linear_sum_assignment(scipy.optimizer)  : The method used is the Hungarian algorithm, also known as the Munkres or Kuhn-Munkres algorithm
def linear_assignment(cost_matrix):
    try:
        import lap
        # print('할당 중 - lapjv 활용')
        _, x, y = lap.lapjv(cost_matrix, extend_cost=True) # y 값에 주목
        return np.array([[y[i],i] for i in x if i >= 0])
        
    except ImportError:        
        from scipy.optimize import linear_sum_assignment
        # print('할당 중 - scipy 활용')
        x, y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x, y)))

In [15]:
# IOUcost matrix 생성
def iou_batch(bb_test, bb_gt):
    """
    From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]

    좌상단(x1, y1), 우하단(x2, y2)
    """
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_test = np.expand_dims(bb_test, 1)
  
    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])                                      
    + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)                                             
    return(o)  

In [16]:
# [x1,y1,x2,y2] 형식의 bbox를 입력받아 center 값 x,y와 크기 s(scale), 각도 r(ratio)을 반환 (shape: 4,1)

def convert_bbox_to_z(bbox):
    """
    Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
    """
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1] # 좌상단, 우하단인데 음수여도 상관없나? 다른곳에서 처리해주나?
    x = bbox[0] + w/2.
    y = bbox[1] + h/2.
    s = w * h    #scale is just area
    r = w / float(h)
    return np.array([x, y, s, r]).reshape((4, 1))

In [17]:
def convert_x_to_bbox(x,score=None):
    """
    Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
    """
    w = np.sqrt(x[2] * x[3])
    h = x[2] / w
    if(score==None):
        return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
    else:
        return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))

In [18]:
class KalmanBoxTracker(object):
    """
    This class represents the internal state of individual tracked objects observed as bbox.
    """
    count = 0
    def __init__(self,bbox): # bbox = dets[i,:] (i : index)
        """
        Initialises a tracker using initial bounding box.
        """
        # define constant velocity model
        # dim_x = n : zeros((n,1)), dim_z = m : np.array([[None]*m]).T
        # 칼만필터 객체 생성
        # dim_x : 상태변수의 수. 만약 두 개체의 속도와 위치를 추척한다면 두 개체의 속도와 위치를 추적하고있다면 4가 된다.
        self.kf = KalmanFilter(dim_x=7, dim_z=4) # x: 상태 변수, z: 측정값(Detection)

        # 상태 천이 행렬, 시간의 변화에 따른 상태의 변화를 야기시키는 행렬 # shape: 7,7
        self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]]) 

        # Measurement function, 측정 기능 # shape: 4,7
        self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

        # Measurement noise matrix  측정 잡음 행렬
        self.kf.R[2:,2:] *= 10.

        #give high uncertainty to the unobservable initial velocities # Covariance matrix 공분산 행렬, 여러 변수와 관련된 공분산을 포함하는 정방형 행렬
        self.kf.P[4:,4:] *= 1000. 
        self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.01 # Process noise matrix 프로세스 잡음 행렬
        self.kf.Q[4:,4:] *= 0.01

        # x = zeros((dim_x, 1)) : zero로 되어있던 x에다 초기값 z 넣어줌
        self.kf.x[:4] = convert_bbox_to_z(bbox) # State estimate vector 상태 측정 벡터

        self.time_since_update = 0

        self.id = KalmanBoxTracker.count    
        KalmanBoxTracker.count += 1

        self.history = []

        self.hits = 0

        self.hit_streak = 0

        self.age = 0

    def update(self,bbox):
        """
        Updates the state vector with observed bbox.
        """

        self.time_since_update = 0

        self.history = []

        self.hits += 1

        self.hit_streak += 1

        self.kf.update(convert_bbox_to_z(bbox))

    def predict(self):
        """
        Advances the state vector and returns the predicted bounding box estimate.
        """

        if((self.kf.x[6]+self.kf.x[2])<=0):
            self.kf.x[6] *= 0.0
        
        # 칼만필터 객체.predict()
        self.kf.predict()

        self.age += 1

        if(self.time_since_update>0):
            self.hit_streak = 0

        self.time_since_update += 1

        self.history.append(convert_x_to_bbox(self.kf.x))

        return self.history[-1]

    def get_state(self):
        """
        Returns the current bounding box estimate.
        """
        return convert_x_to_bbox(self.kf.x)

In [19]:
# D, P들을 acssociation - 헝가리안 알고리즘
# 1차원 배열의 매칭정보[d1,t1,d2,t2, ...], 1차원 배열의 비매칭 D정보[d7, d10, ...], 1차원 배열의 비매칭 T정보[t5, t8, ...] 반환
def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):

    """
    Assigns detections to tracked object (both represented as bounding boxes)

    Returns 3 lists of matches, unmatched_detections and unmatched_trackers
    """
    # trackers가 없으면 바로 끝냄
    if(len(trackers)==0):
        return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)

    iou_matrix = iou_batch(detections, trackers)
    
    # iou_matrix가 존재하면 <-> dets, trks 존재하면
    if min(iou_matrix.shape) > 0:
        
        # iou_matrix 원소를 threshold와 비교하여 크면 1, 아니면 0을 가지는 새로운 matrix 생성
        a = (iou_matrix > iou_threshold).astype(np.int32)
        
        # 어떤 한 행(detection)에서 1이 하나밖에 없고, 한 열(tracking)에서도 1이 하나밖에 없다면 <-> 매칭이 존재한다고 볼 수 있다
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            
            # a에서 원소가 1(!0)인 위치를 [x,y] 꼴의 1차원 리스트로 표현하는 2차원 배열 생성
            # matched_indices 저장 (확정된 매칭들)
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            # 매칭이 없다면 linear_assignment를 통해 할당해준다       
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0,2))
    
    unmatched_detections = []
    
    # detections(이하 D) 중 matched_indices에 없는 D 색출
    for d, det in enumerate(detections):
        if(d not in matched_indices[:,0]):
            unmatched_detections.append(d)

    # trackings(이하 T) 중 matched_indices에 없는 T 색출
    # 항상 len(D)>len(T) 이고 할당 알고리즘에 따라 모두 배정될텐데. unmatched_T 가 있는지 의문이다 # 아래에 보니 threshold 통과 못하면 추가됨
    unmatched_trackers = []
    for t, trk in enumerate(trackers):
        if(t not in matched_indices[:,1]):
            unmatched_trackers.append(t)
    
    #filter out matched with low IOU
    matches = []
    for m in matched_indices:
        # threshold 보다 작으면 unmatched에 추가, 크면 matched에 추가
        if(iou_matrix[m[0], m[1]]<iou_threshold):
            unmatched_detections.append(m[0])
            unmatched_trackers.append(m[1])
        else:
            matches.append(m.reshape(1,2)) # (2,) to (1,2)
    if(len(matches)==0):
        matches = np.empty((0,2),dtype=int)
    else:
        matches = np.concatenate(matches,axis=0) # 2-dim to 1-dim ex)[[0, 1],[1, 2],[2, 0]] to [0,1,1,2,2,0]
    # 매칭정보(indice), non-매칭된 D,T 반환
    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)

In [20]:
class Sort(object):
    def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
        """
        Sets key parameters for SORT
        """
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers = []
        self.frame_count = 0

    def update(self, dets=np.empty((0, 5))):
        # 해당 프레임에서의 dets 정보들이 들어옴
        """
        Params:
          dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...] # dets : detections
        Requires: this method must be called once for each frame even with empty detections (use np.empty((0, 5)) for frames without detections).
        Returns the a similar array, where the last column is the object ID.

        NOTE: The number of objects returned may differ from the number of detections provided.
        """
        # frame_count 증가
        self.frame_count += 1

        # get predicted locations from existing trackers.
        # trks : tracking 정보 담을 배열 초기화
        trks = np.zeros((len(self.trackers), 5))
        to_del = []
        ret = []

        # trks 생성 초기에 trackers는 비어있어서 for문을 돌지않고 지나감
        for t, trk in enumerate(trks):
            # trackers의 원소는 KalmanBoxTracker 객체
            pos = self.trackers[t].predict()[0]            
            trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
        # pos에 nan 값이 있다면 to_del에 t(index) 추가
            if np.any(np.isnan(pos)):
                to_del.append(t)
            
        # compress_rows : 속하는 행에 마스크가 있다면 배제 # masked_invalid : 마스크되어있으면 True, 아니면 False    
        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))    
    
        for t in reversed(to_del):
            self.trackers.pop(t)
        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks, self.iou_threshold)

        # update matched trackers with assigned detections
    
        for m in matched:
            ##################################### 아직 tracker는 [] 일텐데2
            self.trackers[m[1]].update(dets[m[0], :])

        # create and initialise new trackers for unmatched detections
        for i in unmatched_dets:
            trk = KalmanBoxTracker(dets[i,:])            
            self.trackers.append(trk)            
        i = len(self.trackers)
        for trk in reversed(self.trackers):
            d = trk.get_state()[0] # trk bbox의 [0] -> x1. 왜 ?
            # trk이 update를 한 적 없고 등 조건 만족하면,
            if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
                ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
            i -= 1
            # remove dead tracklet
            if(trk.time_since_update > self.max_age): # 수명 초과했으면 trackers에서 제외
                self.trackers.pop(i)
        # 매칭된 결과 반환
        if(len(ret)>0):
            return np.concatenate(ret)
        return np.empty((0,5))

In [21]:
def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(description='SORT demo')
    # Flag 역할 : action = 'store_true' 인 경우, default가 False 이고 실행 시 지정하면 True가 된다
    parser.add_argument('--display', dest='display', help='Display online tracker output (slow) [False]',action='store_true')
    parser.add_argument("--seq_path", help="Path to detections.", type=str, default='data')
    parser.add_argument("--phase", help="Subdirectory in seq_path.", type=str, default='train')
    parser.add_argument("--max_age", # associated detection이 없는 상황이 지속될 시 해당 트랙을 유지하는(살리는) 최대 프레임 수 - 디텍션에서 더 이상 안잡히면 트랙을 없앤다는 뜻
                        help="Maximum number of frames to keep alive a track without associated detections.", 
                        type=int, default=1)
    parser.add_argument("--min_hits", # 트랙이 형성되기 위한 최소한의 associated detections의 수 - 지속적으로 나와야 트랙으로 인식시킨다는 뜻
                        help="Minimum number of associated detections before track is initialised.", 
                        type=int, default=3)
    parser.add_argument("--iou_threshold", help="Minimum IOU for match.", type=float, default=0.3)
    
    #args = parser.parse_args() # 오류 때문에 아래와 같이 수정함 # 터미널에서 .py 실행 시 문제없음
    args, _ = parser.parse_known_args()
    return args

In [22]:
if __name__ == '__main__':
    # all train
    # Argument Parser
    args = parse_args()
    display = args.display # default : False
    phase = args.phase    
    
    total_time = 0.0
    total_frames = 0
    
    # Display (사용 X)
    colours = np.random.rand(32, 3) #used only for display
    if(display):
        print('display is True')
        if not os.path.exists('mot_benchmark'):
            print('\n\tERROR: mot_benchmark link not found!\n\n    Create a symbolic link to the MOT benchmark\n    (https://motchallenge.net/data/2D_MOT_2015/#download). E.g.:\n\n    $ ln -s /path/to/MOT2015_challenge/2DMOT2015 mot_benchmark\n\n')
            exit()
        plt.ion()
        fig = plt.figure()
        ax1 = fig.add_subplot(111, aspect='equal')

    
    # 출력 폴더
    if not os.path.exists('output'):
        os.makedirs('output')
    
    # 패턴 - 영상의 제목을 일반화해서 '*' 로 표시함  
    pattern = os.path.join(args.seq_path, phase, '*', 'det', 'det.txt')

    # seq_dets_fn : Detection 정보가 들어있는 .txt의 위치
    # glob.glob : glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다.
    # 단, 조건에 정규식을 사용할 수 없으며 엑셀 등에서도 사용할 수 있는 '*'와 '?'같은 와일드카드만을 지원한다
    for seq_dets_fn in glob.glob(pattern):        
        # SORT instance 생성
        mot_tracker = Sort(max_age=args.max_age,
                           min_hits=args.min_hits,
                           iou_threshold=args.iou_threshold) #create instance of the SORT tracker
        
        # seq : train 안의 폴더 명
        # seq_dets : (n, 10)의 2차원 넘파이 배열, n = .txt 안의 detections 총 개수, 10 = feature (frame_id, -1, x1, y1, ...)
        seq_dets = np.loadtxt(seq_dets_fn, delimiter=',')

        seq = seq_dets_fn[pattern.find('*'):].split(os.path.sep)[0]
    
        with open(os.path.join('output', '%s.txt'%(seq)),'w') as out_file:
            # print("Step1. Processing %s."%(seq))
            
            # det 중 가장 큰(나중에 나오는) frame_id 만큼 for문 범위 설정 (det이 없다면 영상 전체에 대해 할 필요 없으므로)
            # 근데 이러면 realtime이 아니지 않나?
            for frame in range(int(seq_dets[:,0].max())):
                frame += 1 #detection and frame numbers begin at 1
                
                # dets : 해당하는 frame들의 x1,y1,w,h,c 정보
                dets = seq_dets[seq_dets[:, 0]==frame, 2:7]
                dets[:, 2:4] += dets[:, 0:2] #convert to [x1,y1,w,h] to [x1,y1,x2,y2]
                total_frames += 1
                # 이렇게하면 dets의 마지막 프레임에 대한 정보만 들어가잖아 ?! # 씁...어쩐지 수정완료.
                # print('Step2. frame: {},dets: {}'.format(frame,dets))
                if(display):
                    fn = os.path.join('mot_benchmark', phase, seq, 'img1', '%06d.jpg'%(frame))
                    im =io.imread(fn)
                    ax1.imshow(im)
                    plt.title(seq + ' Tracked Targets')

                # 소요시간 측정
                start_time = time.time()

                trackers = mot_tracker.update(dets)
                # print('Step3. trackers : {}'.format(trackers))
                cycle_time = time.time() - start_time
                total_time += cycle_time

                # d : (5,)의 ndarray [x1,y1,w,h,?] # what is d[4] -> trk_id
                for d in trackers:
                    # print('Step4. Record file: %d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(frame,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]))
                    print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(frame,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]),file=out_file)                    
                    if(display):
                        d = d.astype(np.int32)
                        ax1.add_patch(patches.Rectangle((d[0],d[1]),d[2]-d[0],d[3]-d[1],fill=False,lw=3,ec=colours[d[4]%32,:]))
                

                if(display):
                    fig.canvas.flush_events()
                    plt.draw()
                    ax1.cla()
            # print('current_total_frame: ',total_frames)
        # print('영상 하나 끝. 멈추기')
        
        
    print("Total Tracking took: %.3f seconds for %d frames or %.1f FPS" % (total_time, total_frames, total_frames / total_time))

    if(display):
        print("Note: to get real runtime results run without the option: --display")

current_total_frame:  340
current_total_frame:  865
current_total_frame:  1044
current_total_frame:  1398
current_total_frame:  2235
current_total_frame:  3235
current_total_frame:  4030
current_total_frame:  4630
current_total_frame:  5284
current_total_frame:  5355
current_total_frame:  5500
Total Tracking took: 57.111 seconds for 5500 frames or 96.3 FPS
